# Overview

- It predicts appropriate price from resenting price data.
- It presents a price given another listing data.


# Data
Airbnb listing data in Tokyo http://insideairbnb.com/get-the-data.html
- listings.csv: Detailed Listings data for Tokyo(about 10,000 listings)
- calendar.csv: Detailed Calendar Data for listings in Tokyo(for a year)

In [5]:
# For calendar data
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import luigi
import pickle
import datetime
import time
import locale
import dask
import dask.dataframe as dd
%load_ext memory_profiler

calendar_csv_filename = './data/calendar.csv'

# For listing data
import requests
import json
import statsmodels.api as sm
import os

listings_csv_filename = './data/listings.csv'
neighborhood_data_file = './data/neighborhood.csv'
api_key = 'dummydummydummydummy'
radius = '300'
language = 'en'

First of all, calendar data is processed.

In [2]:
df_calendar = pd.read_csv(calendar_csv_filename)

#########################
# prepare calendar data #
#########################
def data_preparing_calendar(df_calendar):
    use_columns_in_calendar = [
        'listing_id',
        'date',
        'price',
    ]
    df_calendar = df_calendar.loc[:, use_columns_in_calendar]
    df_calendar = df_calendar.dropna()

    # price
    df_calendar['price_amount'] = df_calendar['price'].map(lambda x:float(str(x).replace(',', '').replace('$', '')))

    # date
    locale.setlocale(locale.LC_TIME, 'ja_JP')
    df_calendar['datetime'] = df_calendar['date'].map(lambda x:datetime.datetime.strptime(str(x), '%Y-%m-%d'))
    df_calendar['month'] = df_calendar['datetime'].map(lambda x:x.month)
    df_calendar['day'] = df_calendar['datetime'].map(lambda x:x.day)
    df_calendar['day_of_week'] = df_calendar['datetime'].map(lambda x:x.weekday())
    df_calendar = pd.get_dummies(df_calendar, columns=['month', 'day_of_week'])
    del df_calendar['date']
    del df_calendar['price']
    del df_calendar['datetime']
    
    return df_calendar


%time df_calendar_modified = data_preparing_calendar(df_calendar)

CPU times: user 1min 13s, sys: 2.09 s, total: 1min 15s
Wall time: 1min 16s


In [3]:
print(len(df_calendar_modified))
print(df_calendar_modified.head())

3679189
   listing_id  price_amount  day  month_1  month_2  month_3  month_4  month_5  \
0      776070        6716.0   23        0        0        0        1        0   
1      776070        6716.0   24        0        0        0        1        0   
2      776070        6716.0   25        0        0        0        1        0   
3      776070        6716.0   26        0        0        0        1        0   
4      776070        6716.0   27        0        0        0        1        0   

   month_6  month_7  ...  month_10  month_11  month_12  day_of_week_0  \
0        0        0  ...         0         0         0              0   
1        0        0  ...         0         0         0              0   
2        0        0  ...         0         0         0              0   
3        0        0  ...         0         0         0              0   
4        0        0  ...         0         0         0              0   

   day_of_week_1  day_of_week_2  day_of_week_3  day_of_week_4  day

In [6]:
# ddf_calendar = dd.read_csv(calendar_csv_filename).set_index('listing_id')
# ddf_calendar = dd.from_pandas(df_calendar, npartitions=4).set_index('listing_id')

#########################
# pregare calendar data #
#########################
def data_preparing_calendar_dask(ddf_calendar):
    use_columns_in_calendar = [
#         'listing_id',# it's used as index
        'date',
        'price',
    ]
    ddf_calendar = ddf_calendar.loc[:, use_columns_in_calendar]

    # price
    ddf_calendar['price_amount'] = ddf_calendar['price'].map(lambda x:float(str(x).replace(',', '').replace('$', '')), meta=('x', float))# need to specify type

    # date
    locale.setlocale(locale.LC_TIME, 'ja_JP')
    ddf_calendar['datetime'] = ddf_calendar['date'].map(lambda x:datetime.datetime.strptime(str(x), '%Y-%m-%d'), meta=('x', object))# need to specify type
    ddf_calendar['month'] = ddf_calendar['datetime'].map(lambda x:x.month, meta=('x', object))# need to specify type
    ddf_calendar['day'] = ddf_calendar['datetime'].map(lambda x:x.day, meta=('x', object))# need to specify type
    ddf_calendar['day_of_week'] = ddf_calendar['datetime'].map(lambda x:x.weekday(), meta=('x', object))# need to specify type
    ddf_calendar = ddf_calendar.categorize(columns = ['month', 'day_of_week'])
    ddf_calendar = dd.get_dummies(ddf_calendar, columns=['month', 'day_of_week'])

    del ddf_calendar['date']
    del ddf_calendar['price']
    del ddf_calendar['datetime']
    ddf_calendar = ddf_calendar.dropna()
    ddf_calendar = ddf_calendar.reset_index()
#     ddf_calendar = ddf_calendar.compute(get=dask.multiprocessing.get)
#     >TypeError: The get= keyword has been removed.
#     >Please use the scheduler= keyword instead with the name of
#     >the desired scheduler like 'threads' or 'processes'
    ddf_calendar = ddf_calendar.compute(scheduler='threads')
#     threads:Wall time: 4min 18s(not specify blocksize)
#     Wall time: 4min 40s(not specify blocksize)
    
    return ddf_calendar


ddf_calendar = dd.read_csv(calendar_csv_filename, blocksize='32MB').set_index('listing_id')
%time %memit ddf_calendar_modified = data_preparing_calendar_dask(ddf_calendar)
ddf_calendar = dd.read_csv(calendar_csv_filename, blocksize='64MB').set_index('listing_id')
%time %memit ddf_calendar_modified = data_preparing_calendar_dask(ddf_calendar)
ddf_calendar = dd.read_csv(calendar_csv_filename, blocksize='128MB').set_index('listing_id')
%time %memit ddf_calendar_modified = data_preparing_calendar_dask(ddf_calendar)

peak memory: 1539.93 MiB, increment: 1092.78 MiB
CPU times: user 4min 3s, sys: 49.5 s, total: 4min 53s
Wall time: 4min 15s
peak memory: 2465.60 MiB, increment: 1958.23 MiB
CPU times: user 4min 4s, sys: 53.2 s, total: 4min 57s
Wall time: 4min 19s
peak memory: 2726.47 MiB, increment: 2086.90 MiB
CPU times: user 2min 46s, sys: 11.5 s, total: 2min 58s
Wall time: 2min 51s


In [7]:
print(len(ddf_calendar_modified))
print(ddf_calendar_modified.head())

3679189
   listing_id  price_amount  day  month_4  month_12  month_1  month_11  \
0       35303        4200.0   23        1         0        0         0   
1       35303        4200.0   27        0         1        0         0   
2       35303        4200.0   26        0         1        0         0   
3       35303        4200.0   25        0         1        0         0   
4       35303        4200.0   24        0         1        0         0   

   month_10  month_3  month_2  ...  month_5  month_9  month_8  day_of_week_1  \
0         0        0        0  ...        0        0        0              1   
1         0        0        0  ...        0        0        0              0   
2         0        0        0  ...        0        0        0              0   
3         0        0        0  ...        0        0        0              0   
4         0        0        0  ...        0        0        0              1   

   day_of_week_4  day_of_week_3  day_of_week_2  day_of_week_0  day

In [8]:
def getGooglePlaceData(df_listing):
        # TODO:This should be managed with DB
        neighborhood_data_filepath = neighborhood_data_file + radius + '.pkl'
        if os.path.exists(neighborhood_data_filepath):
            df_neighborhood = pd.read_pickle(neighborhood_data_filepath)
        else:
            df_neighborhood = pd.DataFrame([], columns=['latitude', 'longitude', 'results', 'created'])

        for index, row in df_listing.iterrows():
            # Because the difference is less than 10m, round off to the four decimal places
            latitude_round = round(row.latitude, 4)
            longitude_round = round(row.longitude, 4)

            # find of neighborhood data
            neighborhood = df_neighborhood[(df_neighborhood['latitude'] == latitude_round) & (df_neighborhood['longitude'] == longitude_round)]

            # get only when there is no data
            if neighborhood.empty:
                continue################################ skip only this sumple code
                # if not exist, get data from api
                response = requests.get(self.google_places_api_url + 
                            'key=' + api_key + 
                            '&location=' + str(latitude_round) + ',' + str(longitude_round) + 
                            '&radius=' + radius + 
                            '&language=' + language)
                data = response.json()

                neighborhood = pd.DataFrame([latitude_round, longitude_round, data['results'], time.time()], index=df_neighborhood.columns).T
                df_neighborhood = df_neighborhood.append(neighborhood)

                with open(neighborhood_data_filepath, "wb") as target:
                    pickle.dump(df_neighborhood, target)

            for result in neighborhood.at[0, 'results']:
                column_name = 'neighborhood_' + result['types'][0]
                if not column_name in df_listing.columns:
                    df_listing[column_name] = 0
                df_listing.loc[index, column_name] += 1

        return df_listing

In [9]:
########################
# pregare listing data #
########################
def data_preparing_listing(df_listing):
    use_columns_in_listing = [
        'id',
        'latitude',
        'longitude',
        'property_type',
        'room_type',
        'accommodates',
        'bedrooms',
        'beds',
        'cancellation_policy',
    ]
    df_listing = df_listing.loc[:, use_columns_in_listing]
    df_listing = df_listing.rename(columns={'id': 'listing_id'})
    df_listing = getGooglePlaceData(df_listing)

    del df_listing['latitude']
    del df_listing['longitude']

    # property_type, room_type, cancellation_policy
    df_listing = pd.get_dummies(df_listing, columns=['property_type', 'room_type', 'cancellation_policy'])
    
    return df_listing


df_listing = pd.read_csv(listings_csv_filename)
%time %memit df_listing_modified = data_preparing_listing(df_listing)

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


peak memory: 520.00 MiB, increment: 3.32 MiB
CPU times: user 10.5 s, sys: 198 ms, total: 10.6 s
Wall time: 10.7 s


In [10]:
print(len(df_listing_modified))
print(df_listing_modified.head())

10081
   listing_id  accommodates  bedrooms  beds  property_type_Aparthotel  \
0       35303             1       1.0   1.0                         0   
1      197677             4       1.0   2.0                         0   
2      289597             2       1.0   1.0                         0   
3      370759             2       0.0   1.0                         0   
4      700253             2       1.0   1.0                         0   

   property_type_Apartment  property_type_Barn  \
0                        1                   0   
1                        1                   0   
2                        1                   0   
3                        1                   0   
4                        1                   0   

   property_type_Bed and breakfast  property_type_Boat  \
0                                0                   0   
1                                0                   0   
2                                0                   0   
3                     

In [15]:
########################
# pregare listing data #
########################
def data_preparing_listing_dask(ddf_listing):
    use_columns_in_listing = [
    #     'id',
        'latitude',
        'longitude',
        'property_type',
        'room_type',
        'accommodates',
        'bedrooms',
        'beds',
        'cancellation_policy',
    ]
    ddf_listing = ddf_listing.loc[:, use_columns_in_listing]
    ddf_listing = getGooglePlaceData(ddf_listing)

    del ddf_listing['latitude']
    del ddf_listing['longitude']

    # property_type, room_type, cancellation_policy
    ddf_listing = ddf_listing.categorize(columns = ['property_type', 'room_type', 'cancellation_policy'])
    ddf_listing = dd.get_dummies(ddf_listing, columns=['property_type', 'room_type', 'cancellation_policy'])

    ddf_listing = ddf_listing.reset_index()
    ddf_listing = ddf_listing.rename(columns={'id': 'listing_id'})
    ddf_listing = ddf_listing.compute(scheduler='threads')
    
    return ddf_listing


dtype = {'bedrooms': 'float64',
       'beds': 'float64',
       'review_scores_accuracy': 'float64',
       'review_scores_checkin': 'float64',
       'review_scores_cleanliness': 'float64',
       'review_scores_communication': 'float64',
       'review_scores_location': 'float64',
       'review_scores_rating': 'float64',
       'review_scores_value': 'float64'}
ddf_listing = dd.read_csv(listings_csv_filename, blocksize='32MB', dtype=dtype).set_index('id')
%time %memit ddf_listing_modified = data_preparing_listing_dask(ddf_listing)
ddf_listing = dd.read_csv(listings_csv_filename, blocksize='64MB', dtype=dtype).set_index('id')
%time %memit ddf_listing_modified = data_preparing_listing_dask(ddf_listing)
ddf_listing = dd.read_csv(listings_csv_filename, blocksize='128MB', dtype=dtype).set_index('id')
%time %memit ddf_listing_modified = data_preparing_listing_dask(ddf_listing)

peak memory: 640.87 MiB, increment: 192.34 MiB
CPU times: user 13.9 s, sys: 867 ms, total: 14.8 s
Wall time: 14.4 s


/anaconda3/lib/python3.7/site-packages/dask/core.py:118: DtypeWarning: Columns (61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]


peak memory: 638.93 MiB, increment: 129.93 MiB
CPU times: user 13.3 s, sys: 691 ms, total: 14 s
Wall time: 14.1 s
peak memory: 690.86 MiB, increment: 66.70 MiB
CPU times: user 13.7 s, sys: 704 ms, total: 14.4 s
Wall time: 14.5 s


In [17]:
print(len(ddf_listing_modified))
print(ddf_listing_modified.head())

10081
   listing_id  accommodates  bedrooms  beds  property_type_Apartment  \
0       35303             1       1.0   1.0                        1   
1      197677             4       1.0   2.0                        1   
2      289597             2       1.0   1.0                        1   
3      370759             2       0.0   1.0                        1   
4      700253             2       1.0   1.0                        1   

   property_type_House  property_type_Serviced apartment  \
0                    0                                 0   
1                    0                                 0   
2                    0                                 0   
3                    0                                 0   
4                    0                                 0   

   property_type_Condominium  property_type_Guest suite  property_type_Hut  \
0                          0                          0                  0   
1                          0                

In [41]:
####################
# marge and output #
####################
def marge(df_listing, df_calendar):
    df_intermediate = df_calendar_modified.merge(df_listing, on='listing_id')
    del df_intermediate['listing_id']
    df_intermediate = df_intermediate.dropna()
    
    return df_intermediate

%time %memit df_intermediate = marge(df_listing_modified, df_calendar_modified)

peak memory: 2623.23 MiB, increment: 1838.88 MiB
CPU times: user 6.76 s, sys: 2.83 s, total: 9.59 s
Wall time: 9.69 s


In [42]:
print(len(df_intermediate))
print(df_intermediate.head())

3668969
   price_amount  day  month_1  month_2  month_3  month_4  month_5  month_6  \
0        6716.0   23        0        0        0        1        0        0   
1        6716.0   24        0        0        0        1        0        0   
2        6716.0   25        0        0        0        1        0        0   
3        6716.0   26        0        0        0        1        0        0   
4        6716.0   27        0        0        0        1        0        0   

   month_7  month_8  ...  property_type_Villa  room_type_Entire home/apt  \
0        0        0  ...                    0                          0   
1        0        0  ...                    0                          0   
2        0        0  ...                    0                          0   
3        0        0  ...                    0                          0   
4        0        0  ...                    0                          0   

   room_type_Private room  room_type_Shared room  \
0             

In [ ]:
####################
# marge and output #
####################
def marge_dask(ddf_listing, ddf_calendar):
    ddf_intermediate = ddf_calendar.merge(ddf_listing, on='listing_id', npartitions=2)
    del ddf_intermediate['listing_id']
    ddf_intermediate = ddf_intermediate.dropna()
    
    return ddf_intermediate


%time %memit ddf_intermediate = marge_dask(ddf_listing_modified, ddf_calendar_modified)

In [45]:
print(len(ddf_intermediate))
print(ddf_intermediate.head())

3668969
   price_amount  day  month_4  month_12  month_1  month_11  month_10  month_3  \
0        4200.0   23        1         0        0         0         0        0   
1        4200.0   27        0         1        0         0         0        0   
2        4200.0   26        0         1        0         0         0        0   
3        4200.0   25        0         1        0         0         0        0   
4        4200.0   24        0         1        0         0         0        0   

   month_2  month_6  ...  property_type_Barn  room_type_Private room  \
0        0        0  ...                   0                       1   
1        0        0  ...                   0                       1   
2        0        0  ...                   0                       1   
3        0        0  ...                   0                       1   
4        0        0  ...                   0                       1   

   room_type_Entire home/apt  room_type_Shared room  \
0                

In [49]:
# prepared_filename = './data/prepared.pkl'

# with open(prepared_filename, 'rb') as f:
#     df_intermediate = pickle.load(f)
def create_model(df_intermediate):
    y = df_intermediate['price_amount']
    X = df_intermediate.drop('price_amount', axis=1)

    # 
    model = sm.OLS(y, sm.add_constant(X))
    result = model.fit()
    
    return result

def create_model_L1(df_intermediate):
    y = df_intermediate['price_amount']
    X = df_intermediate.drop('price_amount', axis=1)

    # 
    model = sm.OLS(y, sm.add_constant(X))
    # regularization with elastic net
    result = model.fit_regularized(
        method='elastic_net',
        alpha=1.0,
        L1_wt=1.0,
        start_params=None,
        profile_scale=False,
        refit=False
    )
    
    return result

print('===statsmodels===')
print('<pandas>')
%time %memit model = create_model(df_intermediate)
print('<dask>')
%time %memit model_dask = create_model(ddf_intermediate)

print('===statsmodels===')
print('<pandas>')
%time %memit model_L1 = create_model_L1(df_intermediate)
print('<dask>')
%time %memit model_L1_dask = create_model_L1(ddf_intermediate)

===statsmodels===
<pandas>
peak memory: 6970.64 MiB, increment: 3618.88 MiB
CPU times: user 39.5 s, sys: 18.2 s, total: 57.7 s
Wall time: 57.4 s
<dask>
peak memory: 7120.58 MiB, increment: 4148.00 MiB
CPU times: user 36.6 s, sys: 13.5 s, total: 50.1 s
Wall time: 42.2 s
===statsmodels===
<pandas>
peak memory: 6534.68 MiB, increment: 3047.09 MiB
CPU times: user 16min 56s, sys: 1min 27s, total: 18min 23s
Wall time: 11min 59s
<dask>
peak memory: 4815.23 MiB, increment: 2788.16 MiB
CPU times: user 16min 34s, sys: 1min 16s, total: 17min 50s
Wall time: 10min 47s


In [50]:
print(model.summary())
# print(model_dask.summary())

print(model_L1.params)
# print(model_L1_dask.params)

                            OLS Regression Results                            
Dep. Variable:           price_amount   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.030
Method:                 Least Squares   F-statistic:                     2069.
Date:                   月, 08  7 2019   Prob (F-statistic):               0.00
Time:                        19:01:32   Log-Likelihood:            -4.5657e+07
No. Observations:             3668969   AIC:                         9.131e+07
Df Residuals:                 3668913   BIC:                         9.131e+07
Df Model:                          55                                         
Covariance Type:            nonrobust                                         
                                                      coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------